## 文本分类步骤
- 划分数据集
- 对标题和正文分词和去停用词
- 计算tf-idf等特征
- 构建分类器

### 从ES取出带标签的数据，分词，并dump到本地

In [1]:
import sys
sys.path.append('..')
import esProxy
from analyzer import Analyzer
import pickle, random

# 从ES导出带标签的新闻数据
sougouNews = esProxy.getDataFromEs()
print('成功导出新闻数据：size=%d' % (len(sougouNews)))

def featurelize(sougouNews, fields=['doc_title'], analyzer=Analyzer()):
    """
    返回标签和分词后的特征
    """
    tokens = []
    for doc in sougouNews:
        dic = {}
        for field in fields:
            dic[field] = analyzer.cutAndFilter(doc[field])
        # 添加新闻类别
        dic['doc_type'] = doc['doc_type']
        tokens.append(dic)
    return tokens

# 对新闻标题进行分词，得到带分词的新闻数据
tokenSougouNews = featurelize(sougouNews, fields=['doc_title'], analyzer=Analyzer())
print('完成对新闻标题的分词')

# 将分词后的结果dump到本地
with open('tokenSougouNews.pk', 'wb') as f:
    f.truncate()
    pickle.dump(tokenSougouNews, f)
print('成功将分词后的数据dump到本地')

# 划分训练集和测试集
random.shuffle(tokenSougouNews)
trainPercent = 0.8
# dump训练集
with open('tokenSougouNews-train.pk', 'wb') as f:
    f.truncate()
    pickle.dump(tokenSougouNews[:int(trainPercent*len(tokenSougouNews))], f)
print('成功dump训练集到本地：size=%d' % (int(trainPercent*len(tokenSougouNews))))
    
# dump测试集
with open('tokenSougouNews-test.pk', 'wb') as f:
    f.truncate()
    pickle.dump(tokenSougouNews[int(trainPercent*len(tokenSougouNews)):], f)
print('成功dump测试集到本地：size=%d' % (len(tokenSougouNews) - int(trainPercent*len(tokenSougouNews))))

tokenSougouNews[0]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\YDQing\AppData\Local\Temp\jieba.cache


成功导出新闻数据：size=103320


Loading model cost 1.042 seconds.
Prefix dict has been built succesfully.


完成对新闻标题的分词
成功将分词后的数据dump到本地
成功dump训练集到本地：size=82656
成功dump测试集到本地：size=20664


{'doc_title': ['南', '天', '信息', '管理层', '增持', '86', '万股'], 'doc_type': 'IT'}

### tf-idf + bayes分类

#### 加载训练数据和测试数据

In [1]:
import pickle

with open('tokenSougouNews-train.pk', 'rb') as f:
    trainData = pickle.load(f)
trainX = [dict(doc_title=' '.join(d['doc_title'])) for d in trainData]
trainY = [d['doc_type'] for d in trainData]
print('train size=%d' % (len(trainX)))
    
with open('tokenSougouNews-test.pk', 'rb') as f:
    testData = pickle.load(f)
testX = [dict(doc_title=' '.join(d['doc_title'])) for d in testData]
testY = [d['doc_type'] for d in testData]
print('test size=%d' % (len(testX)))

train size=82656
test size=20664


#### 将文本tf-idf向量化

In [2]:
import sklearn.feature_extraction.text as text
import numpy as np

class TfidfVectorizor(object):
    def __init__(self, fields):
        """
        fields: 需要向量化的属性
        """
        self.fields = fields
        self.tfidfVectorizors = dict()
        for field in fields:
            self.tfidfVectorizors[field] = text.TfidfVectorizer()
    
    def fit(self, X, y=None):
        for field in self.fields:
            docs = [x[field] for x in X]
            self.tfidfVectorizors[field].fit(docs, y)
        return self
    
    def transform(self, X):
        """
        将每个属性向量化后，拼接成一个向量
        """
        vectors = None
        for i, field in enumerate(self.fields):
            docs = [x[field] for x in X]
            vector = self.tfidfVectorizors[field].transform(docs)
            vectors = np.hstack(vectors, vector) if i > 0 else vector
        return vectors

#### 构建分类器

In [3]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

nbClf = Pipeline([('tfidfVectorizor', TfidfVectorizor(['doc_title'])),
                 ('multinomialNB', MultinomialNB())])
nbClf.fit(trainX, trainY)

# 计算误差
trainAcc = accuracy_score(trainY, nbClf.predict(trainX))
testAcc = accuracy_score(testY, nbClf.predict(testX))
print('tfidf+multiNB: trainAcc=%f, testAcc=%f' % (trainAcc, testAcc))

svmClf = Pipeline([('tfidfVectorizor', TfidfVectorizor(['doc_title'])),
                 ('svm', LinearSVC())])
svmClf.fit(trainX, trainY)

# 计算误差
trainAcc = accuracy_score(trainY, svmClf.predict(trainX))
testAcc = accuracy_score(testY, svmClf.predict(testX))
print('tfidf+svm: trainAcc=%f, testAcc=%f' % (trainAcc, testAcc))

C:\Anaconda34\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


tfidf+multiNB: trainAcc=0.867886, testAcc=0.821235
tfidf+svm: trainAcc=0.981018, testAcc=0.895906
